In [29]:
from langchain_groq import ChatGroq

In [30]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [31]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2904059/software-development-engineer-in-test-2025-us")
page_data = loader.load().pop().page_content
print(page_data)

Software Development Engineer in Test – 2025 (US) - Job ID: 2904059 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsDiversity, equity, and inclusionInterview tipsLeadership principlesWorking at AmazonFAQ×Software Development Engineer in Test – 2025 (US)Job ID: 2904059 | Amazon.com Services LLCApply nowView transcriptDESCRIPTIONDo you want to write secure, stable, testable and maintainable code, as well as test it? Do you enjoy working on automation frameworks in a team environment?At Amazon, we hire the best minds in technology to innovate on behalf of our customers. The focus we have on our customers is why we are one of the world’s most beloved brands – customer obsession is part of our company DNA. Software Development-Test Engineers (SDE-T) use cutting-edge technology to solve complex problems and get to see the impact of their work first-hand.As an SDE-T yo

In [32]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

Note: you may need to restart the kernel to use updated packages.


{'role': 'Software Development Engineer in Test',
 'experience': "0-2 years (recent graduate or currently working towards a Bachelor's or master's degree)",
 'skills': ['Object-oriented programming language (e.g., Java, Python)',
  'Software engineering best practices',
  'Software test techniques and strategy',
  'Test automation framework or tools (e.g., Junit, TestNG, Selenium)'],
 'description': 'Collaborate with experienced cross-disciplinary teams to use technology to validate and verify software. Invent, refine and develop solutions to help lead fundamental changes in the industry. Build tools and automation frameworks that enable QA Engineers to easily automate tests. Troubleshoot, research and resolve defects.'}

In [34]:
type(json_res)

dict

In [35]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [36]:

import uuid
import chromadb
import socket

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        try:
            collection.add(documents=row["Techstack"],
                           metadatas={"links": row["Links"]},
                           ids=[str(uuid.uuid4())])
        except socket.gaierror as e:
            print(f"Network error occurred: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

In [39]:
job = json_res
job['skills']

['Object-oriented programming language (e.g., Java, Python)',
 'Software engineering best practices',
 'Software test techniques and strategy',
 'Test automation framework or tools (e.g., Junit, TestNG, Selenium)']

In [40]:
job

{'role': 'Software Development Engineer in Test',
 'experience': "0-2 years (recent graduate or currently working towards a Bachelor's or master's degree)",
 'skills': ['Object-oriented programming language (e.g., Java, Python)',
  'Software engineering best practices',
  'Software test techniques and strategy',
  'Test automation framework or tools (e.g., Junit, TestNG, Selenium)'],
 'description': 'Collaborate with experienced cross-disciplinary teams to use technology to validate and verify software. Invent, refine and develop solutions to help lead fundamental changes in the industry. Build tools and automation frameworks that enable QA Engineers to easily automate tests. Troubleshoot, research and resolve defects.'}

In [41]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Satyendra, a Data Science at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Development Engineers in Test for Your Team

Dear Hiring Manager,

I came across the job description for a Software Development Engineer in Test at your esteemed organization, and I am excited to introduce AtliQ, an AI & Software Consulting company that can fulfill your needs. With our expertise in software engineering, test automation, and automation frameworks, we are confident that our team can collaborate with yours to drive fundamental changes in the industry.

Our team of skilled engineers has extensive experience in object-oriented programming languages such as Java and Python, software engineering best practices, and software test techniques and strategy. We have worked with various test automation frameworks and tools like Junit, Testng, and Selenium, and have a proven track record of delivering high-quality solutions.

At AtliQ, we have empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction,